In [1]:
import os

embedding_size = 128
hidden_size = 64
batch_size = 4
output_size = 10 # It is known that Willow Cove features 10 "ports"
num_heads = 2

learning_rate = 0.001

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "backend:cudaMallocAsync"

In [2]:
import model.utils
import torch
from torch_geometric.loader import DataLoader

dataset = model.utils.BasicBlockDataset("data/i5_1135g7.pb")
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [3]:
from model.GraphEncoder import GATEncoder
from model.Predictor import Predictor
import torch_geometric

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

encoder = GATEncoder(dataset.num_opcodes, embedding_size, hidden_size, dataset.num_opcodes).to(device)
model = Predictor(encoder, hidden_size, output_size).to(device)

In [4]:
from model.model import train

checkpoint_dir = "checkpoints/tgl"
checkpoint_freq = 10
num_epochs = 200

if torch.cuda.is_available():
    torch.cuda.empty_cache()

train(model, device, loader, num_epochs, batch_size, learning_rate, checkpoint_dir, checkpoint_freq)

  0%|          | 0/1253600 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.save(model, "trained_models/tgl.pt")

In [ ]:
choice = dataset[140]
bb, m, raw = choice

print(bb.x)

input_sequence = bb.x.to(device)
edge_index = bb.edge_index.to(device)

out, _ = model(input_sequence, edge_index)
res = out.to("cpu").detach().numpy()
model.utils.print_port_pressure_table(res[1:], raw["source"])
print(model.utils.estimate_cycles(out))
print(m)